### Cette méthode n'est pas bonne, car le dictionnaire n'est pas bonne, de plus, quand on fait stemmer avec ce dictionnaire, il se trompe des mots, par exemple: génération---> géner qui est trompé par généreux, donc, il trouve les synonymes sont grand, beaucoup...

In [82]:
lis_= ['grand', 'gross', 'élevé', 'haut', 'haute']
for word in lis_:
    print(word, ' ', stemmer.stem(word))

grand   grand
gross   gross
élevé   élev
haut   haut
haute   haut


In [75]:
'faut' in STOPWORDS

False

In [2]:
"""
les symboles particulières comme ? ! et . sont supprimé dans la question.
Il faut peut être returner une réponse spécifique si il rencontre plus 1 mot qu'il existe pas dans le corpus
Nous suprimmons également:
  1. Mot qui contient une seule lettre(qui sont suivante des erreurs)
  2. Une fonction unique pour nettoyage tous les données dans train-test et chat
Nous allons traiter tous les mots clés suivant :
km, vitesse, carburant, huile, position, batterie, pression_pneu. Il reste trajet et vin
  
"""
#import sys
#!{sys.executable} -m pip uninstall gtts

#from gtts import gTTS
#from playsound import playsound
#import speech_recognition
from nltk.corpus import stopwords
STOPWORDS = stopwords.words('french')
from nltk.stem import *
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french", ignore_stopwords = False)

import re
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os
import math
import time
from dateparser.search import search_dates
import pandas as pd

def clearing_word(word):
    word = re.sub('\x8e', 'é', word)
    word = re.sub('\x88', 'à', word)
    word = re.sub('\x9d', 'ù', word)
    word = re.sub('\x8f', 'è', word)
    word = re.sub('\x9e', 'û', word)
    word = re.sub('\x90', 'ê', word)
    word = re.sub('\x99', 'ô', word)
    word = re.sub('\x94', 'î', word)
   # word = re.sub('\x8f', 'è', word)
    word = re.sub('\x8d', 'ç', word)
    word = re.sub('õ', '', word)
    word = re.sub('Ê', '', word)
    word = re.sub('[?,.,!, \,,  %]', '', word)
    if word == 'û' or word == 'v' or word == 'é':
        word = ''
    if word =="2017êles":
        word = "2017"
    if "ênox" in word:
        word ="nox"
    return word

def clear_line(pharse, stop_word):
    """
    Arg: string
    Return: string
    """
    #clean_data = []
    pharse_ =[]
    for words in pharse.strip().strip('?').strip('.').strip('!').lower().split(' '):
        for word in words.split('-'):
            for word_ in word.split('\''):
                word_ = clearing_word(word_)
               # word_ = stemmer.stem(word_)
        if word_ not in stop_word and len(word_) >1:       
            pharse_.append(word_)        
    return ' '.join(pharse_)


def transformed_line(line, dicts):
    line_to_return, word_not_in_list = [], []
    for word in line.split():
        if word in LIST_OF_WORDS:
            line_to_return.append(word)
        else:
            word_not_in_list.append(word)

    for word in word_not_in_list:
        for Line in dicts:
            if word in Line and word not in line_to_return:
                line_to_return.append(Line[0])
                break
    return ' '.join(line_to_return), len(line.split())-len(line_to_return)


def prepareData(PAIRS):
    pairs_trains, pairs_tests = [], []
   # PAIRS.extend(pairs1)
    index_train = random.sample(range(len(PAIRS)), int(len(PAIRS)*0.80))
    for i in range(len(PAIRS)):
        if i in index_train:
            pairs_trains.append(PAIRS[i])
        else:
            pairs_tests.append(PAIRS[i])
    print("Read %s sentence pairs of training set" % len(pairs_trains))
    print("Read %s sentence pairs of test set" % len(pairs_tests))
    return pairs_trains, pairs_tests

def get_all_convos(stopwords, file):
    
    convos = []
    #file = 'convos27juin.txt'
    #file1 = 'chatbot_tout_corpus.txt'
    #file2 = 'chatbot_tout_corpus_13juin.txt'
    #file3 = 'convos_120_nettoye.txt' # cette fichier viens d'etre ajouter
    liste_file = [file]
    list_of_word = []
    for file in liste_file:
        with open(file) as f:
            i=0
            for line in f:
                if i%2==0:
                    question = str(line)
                    if '++++' in question:
                         question = question[9:]
                    question = clear_line(question, stopwords)
                else:
                    answer = str(line)
                    if '++++' in answer:
                        answer = answer[9:].strip()
                    convos.append([question, answer])
                i+=1
        convos_to_return = []
        for pair in convos:
            if len(pair[0]) !=0:
                convos_to_return.append(pair)
        f.close()
    return convos_to_return

def get_list_words(convos):
    list_words =[]
    for pair in convos:
        for word in pair[0].split():
            if word not in list_words:
                list_words.append(word)
    print('There are {} words in corpus:'.format(len(list_words)))
    return list_words
                

def simulation_pairs():
    
    DIRECT_VARIABLE = ['vitesse', 'batterie', 'position', 'km']
    TIME = []
    GEOGRAPHY = ['Lyon', 'hors lyon', 'paris', 'l\'étranger','france' , 'hors la france']
    BORDE = {'MAX': ['plus grand', 'plus grande', 'plus grands', 
         'plus grandes',
         'plus vite', 'max', 'maximum', 'maximal', 'maximaux', 'maximale',
        'plus haut', 'plus haute', 'plus hautes', 'plus hauts',  
        'plus élevé', 'plus élevée', 'plus élevés', 'plus élevées'], 
         'MIN':['moins grand', 'moins grande', 'moins grands','moins grandes',
        'plus petit', 'plus petite', 'plus petits', 'plus petites','plus faible', 'plus faibles', 
         'min', 'minimal', 'minimale', 'minimales', 'minimaux','moins vite', 
         'moins élevé', 'moins élevée', 'moins élevés', 'moins élevées'],
        'MOYENNE': ['moyenne', 'moyen', 'moyennement']
        }
    COMPLEX_ANALYSIS = ['problème', 'problèmes', 'erreurs', 'erreur', 'danger']
    PAIRS = []
    question_for_simulation = ['quels véhicules ont', 'quel véhicule', 'quelle voiture', 'quelles voitures']
    reponse_for_simulation = 'véhicule'
    for borde in BORDE:
        for word in BORDE[borde]:
            for direct_variable in DIRECT_VARIABLE:
                for question_f_s in question_for_simulation:
                    if direct_variable !='position':
                        question= question_f_s+' '+direct_variable + ' '+ word
                        key0 = '#id#'
                        key1 = '#'+direct_variable+'#'
                        key2 = '#'+borde +'#'
                        reponse = reponse_for_simulation +' '+key0+ ' '+ key1+ ' '+ key2
                        PAIRS.append([question,reponse])

    question_for_simulation = ['', 'quel véhicule a', 'quelle voiture a', 'quelles voitures ont', 'il y a']
    reponse_for_simulation = 'véhicule' 
    for direct_variable in DIRECT_VARIABLE:
        for complex_analysis in COMPLEX_ANALYSIS:
            for question_f_s in question_for_simulation:
                #if direct_variable !='position':
                      question= question_f_s+' '+complex_analysis + ' '+ direct_variable
                      key0 = '#id#'
                      key1 = '#'+direct_variable+'#'
                      key2 = '#'+ complex_analysis +'#'
                      reponse = reponse_for_simulation +' '+key0+ ' '+ key2 +' '+ key1
                      PAIRS.append([question,reponse])
    return PAIRS

def clear_pairs_trains(pairs_trains):
    questions = []
    index = []
    for i in range(len(pairs_trains)):
        if  pairs_trains[i][0] not in questions:
            index.append(i)
            questions.append(pairs_trains[i][0])
    return [pairs_trains[i] for i in index]

def tf_idf(word, doc, méthode):
    tf, df = 0, 0
    if word in str(doc):
        tf = 1
    for doc_ex in QUESTION_TO_TRAINS:
        if word in str(doc_ex):
            df+=1
    if méthode  == 'normal':
        if df>0:
            return tf*math.log(len(QUESTION_TO_TRAINS)/(df), 10)
        else:
            return 0
    if méthode == 'probabiliste':
        if df>0:
            return tf*math.log((len(QUESTION_TO_TRAINS)-df)/(df), 10)
        else:
            return 0
    if méthode == 'lissé':
        if df>0:
            return tf*(1+math.log(len(QUESTION_TO_TRAINS)/df, 10))
        else:
            return tf
    if méthode == 'probabiliste_lissé':
        if df>0:
            return tf*(1+math.log((len(QUESTION_TO_TRAINS)-df)/(df), 10))
        else:
            return tf
    
    
def dict_doc_score(doc, méthode = 'normal'):   
    return {word: tf_idf(word, doc, méthode) for word in doc.split()}
   
    
def score_by_new_doc(new_doc, doc, DICT, méthode):
    
    if méthode != 'Lucene':
        DICT_of_methode = DICT[méthode]
        score = 0
        for word in new_doc.split():
            if word in DICT_of_methode[doc]:
                 score += DICT_of_methode[doc][word]
        return score
    else:
        DICT_of_methode = DICT[méthode]
        score = 0
        for word in new_doc.split():
            if word in DICT_of_methode[doc]:
                 score += DICT_of_methode[doc][word]
        coord = 1.5*len([word for word in new_doc if word in doc])
        return  coord*score
    
def get_key_list(pairs_trains):
    
    KEY_LIST = []
    for pair in pairs_trains:
        for word in pair[1].split():
            if '#' in word and word not in KEY_LIST:
                word = re.sub('#', '', word)
                KEY_LIST.append(word)
    return KEY_LIST
        

def find_right_index(new_doc,DICT, méthode = 'normal'):
    
    index=0
    biggest_score = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[0],DICT, méthode)
    for i in range(len(QUESTION_TO_TRAINS)):
        boolean1 = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i], DICT, méthode) >biggest_score
        boolean2 = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i], DICT, méthode) == biggest_score
        boolean3 = len(QUESTION_TO_TRAINS[index]) >len(QUESTION_TO_TRAINS[i])
        if boolean1 or (boolean2 and boolean3):
            biggest_score = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i],DICT, méthode)
            index = i
    précision, rappel = precision_recall(new_doc, QUESTION_TO_TRAINS[index])
    if biggest_score ==0 or précision <0.3 or rappel<0.3:
        index = len(QUESTION_TO_TRAINS)-1
    return index

def test(DICT, méthode, train =False):
    
    if not train:
        index_to_print = random.sample(range(len(pairs_tests)), 20)
        #loss_total = 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            #loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_tests[i][1])
            #loss_total +=loss
            if i in index_to_print:
                print(pairs_tests[i])
                print(pairs_trains[index][1])
        print('-'*80)
       # print('ACCURACY=', 1-loss_total/len(pairs_tests))
    if  train:
        #loss_total = 0
        #index_to_print = random.sample(range(len(pairs_trains)), 20)  
        list_of_bad_prediction = []
        i0 = 0
        for i in range(len(pairs_trains)):
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            #loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_trains[i][1])
            #loss_total +=loss
            if index != i :
                i0 +=1
                print(pairs_trains[i])
                print(pairs_trains[index][1])
        print('-'*80)
        #print('ACCURACY=', 1-loss_total/len(pairs_trains))
        print('Il y a {} erreurs d\'indexes parmis {} prédictions'.format(i0, len(pairs_trains)))

def test_without_print(DICT, méthode, train =False):
    
    if not train:
        loss_total = 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_tests[i][1])
            loss_total +=loss
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_tests))
    if  train:
        loss_total = 0
        index_to_print = random.sample(range(len(pairs_trains)), 20)   
        for i in range(len(pairs_trains)):
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_trains[i][1])
            loss_total +=loss
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_trains))
        
        
def chat(méthode):
    
    print('Bonjour, C\'est le bot d\'Avicen, pose tes questions, s\'il te plaît!')
    path = 'processed/chat_tfidf.txt'
    f =  open(path, 'a+') 
    while True:
            line = str(input('Vous: '))
            LINE = clear_line(line, STOPWORDS)
            LINE, unknown_word = transformed_line(LINE, DICTIONNARY)
            print('Ligne tranformée: ',LINE)
            if unknown_word > 0:
                print('Désolé, je ne comprends pas ta question, peux tu la reformuler s\'il te plaît?')
                continue
            if len(line) !=0:
                index= find_right_index(LINE, DICT, méthode)
                print('bot: ', pairs_trains[index][1])
                f.write('VOUS ++++ '+line+'\n')
                f.write('BOT ++++ '+pairs_trains[index][1]+'\n')
            
            else:
                f.close()
                break 

def precision_recall(lstcomp, lstref):
    
    card_intersec = 0.0 # force à utiliser la division non entière
    for t in set(lstcomp) :
        card_intersec += min(lstref.count(t), lstcomp.count(t))
    if len(lstcomp)==0:
        precision =1
    else:
        precision = card_intersec/len(lstcomp)
    if len(lstref)==0:
        rappel = 1
    else:
        rappel = card_intersec/len(lstref)
    return (precision, rappel)


def test_with_precision_recall(DICT, méthode, train = False):
    
    if not train:
        total_precision, total_recall = 0, 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            precision, recall = precision_recall\
            (pairs_tests[i][1], pairs_trains[index][1])
            total_precision  += precision
            total_recall     += recall
        print('-'*80)
        total_precision = total_precision/len(pairs_tests)
        total_recall = total_recall/len(pairs_tests)      
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        print('PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} METHODE = {}'\
              .format(total_precision, total_recall,F, méthode))
    if  train:
        total_precision, total_recall = 0, 0
        for i in range(len(pairs_trains)):          
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            precision, recall = precision_recall\
            (pairs_trains[i][1], pairs_trains[index][1])
            total_precision  += precision
            total_recall     += recall
        print('-'*80)
        total_precision = total_precision/len(pairs_trains)
        total_recall = total_recall/len(pairs_trains)
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        print('PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} METHODE = {}'\
              .format(total_precision, total_recall, F, méthode))
        
            
def dict_of_all_score(METHODE):
    """
    return score of all word in all doc"""
    DICT = {}
    for méthode in METHODE:
        dict_for_this_method ={}
        for doc in QUESTION_TO_TRAINS:
            dict_for_this_method[doc]=dict_doc_score(doc, méthode)
        DICT[méthode] = dict_for_this_method
    return DICT
        
def chatting_with_corpus():


    path = 'processed/chat_tfidf.txt'
    i = 0
    questions = []
    with open(path, 'r') as f:
        for line in f:
            if i%2 ==0:
                line = line[9:]
                line = line.lower().strip()
                questions.append(line)
            i +=1
    f.close()
    questions_clean = []
    for question in questions:
        if question not in questions_clean:
            questions_clean.append(question)
    for line in questions_clean:
                LINE = clear_line(line, STOPWORDS)   
                index = find_right_index(LINE, DICT, METHODE[0])
                print('YOU: ', line)
                print('BOT: ', pairs_trains[index][1])
                print()
        
def chatting_with_test_corpus():
    
    path = 'test.txt'
    questions = []
    with open(path, 'r') as f:
        for line in f:
                line = line.lower().strip()
                questions.append(line)
    f.close()
    questions_clean = []
    for question in questions:
        if question not in questions_clean:
            questions_clean.append(question)
    for line in questions_clean:
                LINE = clear_line(line, STOPWORDS) 
                LINE, _ = transformed_line(LINE, DICTIONNARY)
                index = find_right_index(LINE, DICT, METHODE[0])
                print('YOU: ', line)
                print('BOT: ', pairs_trains[index][1])
                print()
            

def get_all_and_all_convos():
    convos = []
    file1 = 'convos27juin_test.txt'
    file2 = 'convos27juin_train.txt'
    files = [file1, file2]
    for file in files:
        with open(file, 'r') as f:
            i=0
            for line in f:
                if i%2==0:
                    question = line.strip()
                    if '++++' in question:
                         question = question[9:]
                else:
                    answer = line.strip()
                    if '++++' in answer:
                        answer = answer[9:]
                    convos.append([question, answer])
                i+=1
        f.close()
    return convos

def clear_all_convos(all_convos, name):
    """
    Supprimer tous les couples où les questions sont les mêmes"""
    index_of_convos = []
    question = []
    index =0
    for pair in all_convos:
        if pair[0] not in question:
            index_of_convos.append(index)
            question.append(pair[0])
        index +=1
    convos = [all_convos[index] for index in index_of_convos]
    print('Il y a {} de couples de  {}'.format(len(convos), name))
    #print('nous avons sumpprimer {} couples parmis {} couples'.format(len(all_convos)-len(index_of_convos),index))
    return convos

def save_convos_in_file(all_convos):
    
    index_of_train = random.sample(range(len(all_convos)), int(len(all_convos)*0.80))
    with open('convos6juillet_train.txt', 'w') as f:
        for index in index_of_train:
            pair = all_convos[index]
            f.write('YOU ++++ '+ pair[0]+'\n')
            f.write('BOT ++++ '+ pair[1]+'\n')
    f.close()
    with open('convos6juillet_test.txt', 'w') as f:
        for index in range(len(all_convos)):
            if index not in index_of_train:
                pair = all_convos[index]
                f.write('YOU ++++ '+ pair[0]+'\n')
                f.write('BOT ++++ '+ pair[1]+'\n')
    f.close()
    len_test = len(all_convos)- len(index_of_train)
    print('save {} convos in train and {} convos in test'.format(len(index_of_train), len_test))
    
def test_to_find_error(dict):
    
    index_of_error_prediction = []
    index_of_true_pair = []
    i = 0
    for pair in pairs_trains:
        index = find_right_index(pair[0],dict,  'normal')
        if index != i:
            index_of_error_prediction.append(index)
            index_of_true_pair.append(i)
        i +=1
    for i in range(len(index_of_error_prediction)):
        index1 = index_of_error_prediction[i]
        index2 = index_of_true_pair[i]
        score1 = score_by_new_doc(pairs_trains[index1][0], pairs_trains[index2][0],dict, 'normal')
        score2 = score_by_new_doc(pairs_trains[index2][0], pairs_trains[index2][0],dict, 'normal')
        print('False return question--- {} ---with score {}'.format(pairs_trains[index1][0], score1))
        print('True return question--- {} ---with score {}'.format(pairs_trains[index2][0], score2))

def print_all_errors_on_test(dict):
    for pair in pairs_tests:
        question = clear_line(pair[0], STOPWORDS)
        index = find_right_index(question, dict, 'normal')
        print(pair)
        print(pairs_trains[index])
        print()
        
def clear_all_trains_test_and_save_in_file(convos_train, convos_test):
    
    convos_train.extend(convos_test)

    
def construct_dict(dicts):
    
    in_file = open(os.path.join('processed', 'dictionnary.txt'), 'r')
    lines = []
    words = []
    LINES_to_return = []
    for line in in_file:
        line = line.strip().split('|')
        line = [stemmer.stem(word) for word in line]
        lines.append(line)
    for line in lines:
        for index in range(len(dicts)):
            word = dicts[index]
            if str(word) in line and word not in words:
                line.insert(0,str(word))
                index1 = min(10, len(line))
                LINES_to_return.append(line[:index1])
                words.append(word)
    for word in dicts:
        boolean = False
        for line in LINES_to_return:
            if word in line:
                boolean = True
        if not boolean:
             LINES_to_return.append([word])
    return LINES_to_return

    
if __name__=='__main__':
    try:
        pairs_trains_
    except NameError :
        pairs_trains = get_all_convos(STOPWORDS, 'convos27juin_train_smaller.txt')
        pairs_tests = get_all_convos(STOPWORDS, 'convos27juin_test_smaller.txt')
        pairs_trains = clear_all_convos(pairs_trains, 'train')
        pairs_tests = clear_all_convos(pairs_tests, 'test')
        #pairs_trains.extend(pairs_tests)
        LIST_OF_WORDS = get_list_words(pairs_trains)
        KEY_LIST = get_key_list(pairs_trains)
        QUESTION_TO_TRAINS = []
        for pairs in pairs_trains:
            QUESTION_TO_TRAINS.append(pairs[0])
        METHODE = ['normal','lissé', 'probabiliste','probabiliste_lissé']
        #METHODE = ['normal']
        DICT = dict_of_all_score(METHODE)  
        DICTIONNARY = construct_dict(LIST_OF_WORDS)
print('on train:')
for méthode in METHODE:
    test_with_precision_recall(DICT, méthode, train = True)
print('On test:')
for méthode in METHODE:
    test_with_precision_recall(DICT, méthode, train = False)

Il y a 766 de couples de  train
Il y a 195 de couples de  test
There are 387 words in corpus:
on train:
--------------------------------------------------------------------------------
PRECISION = 1.00000,  RECALL = 1.00000, F_MESURE = 1.000 METHODE = normal
--------------------------------------------------------------------------------
PRECISION = 1.00000,  RECALL = 1.00000, F_MESURE = 1.000 METHODE = lissé
--------------------------------------------------------------------------------
PRECISION = 0.99559,  RECALL = 0.99846, F_MESURE = 0.997 METHODE = probabiliste
--------------------------------------------------------------------------------
PRECISION = 1.00000,  RECALL = 1.00000, F_MESURE = 1.000 METHODE = probabiliste_lissé
On test:
--------------------------------------------------------------------------------
PRECISION = 0.84177,  RECALL = 0.83406, F_MESURE = 0.838 METHODE = normal
--------------------------------------------------------------------------------
PRECISION = 0.

In [7]:
for i in range(20):
    dict_ = random.choice(DICTIONNARY)
    print(dict_)


['thématiques']
['ouverture']
['parcourus']
['dernière']
['combien', 'combien', 'quel nombr']
['huile']
['indécis', 'angoiss', 'peur', 'craint', 'anxiet', 'détress', 'désarroi', 'épouv', 'frayeur', 'tourment']
['co2']
['france']
['monde']
['obtenir']
['contact', 'entent', 'accord', 'convent', 'compliqu', 'connivent', 'collus', 'coalit', 'concord', 'amiti']
['énergie']
['disponible']
['fort', 'parc', 'asil', 'refug', 'retrait', 'cachet', 'gît', 'tani', 'antre', 'toit']
['grande']
['kilomètrage']
['ampoules']
['temps']
['normal', 'abord', 'accueil', 'proch', 'voisin', 'accost', 'compréhensibl', 'atteign', 'joignabl', 'intelligibl']


In [16]:
for dict_ in DICTIONNARY:
    if 'bien' in dict_:
        print(dict_)

['bien', 'moyen', 'profus', 'richess', 'foison', 'pléthor', 'délug', 'opulent', 'affluenc', 'exuber']
['avoir', 'bien', 'avoir', 'capital', 'ressourc', 'affair', 'entrepren', 'diligent', 'travailleur', 'vif']
